In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rcParams
import numpy as np
import os

In [2]:
plt.rc('font', family='Malgun Gothic')

In [3]:
station_list=[]
for i in os.listdir("./station_list"):
    station_list.append(i)

In [4]:
def read_csv_only_one(station_name, year, month):
    if year != '2020':
        df1 = pd.read_csv(f'station_list/{station_name}/{year}/{station_name}_{year}_{month}.csv')
    else:
        df1 = pd.read_csv(f'station_list/{station_name}/{year}/{station_name}_2020_1.csv')
    return df1

In [5]:
def make_pmList(df):
    pm10=[]
    pm2_5=[]
    for i in df['pm10']:
        pm10.append(i)
    for i in df['pm2.5']:
        pm2_5.append(i)
    return pm10, pm2_5

In [56]:
df1 = read_csv_only_one('Yongsan_Hangandaero','2019','1')
df2 = read_csv_only_one('Yongsan_Hannamdaero','2019','1')

In [64]:
df1.head()

,time,pm10,pm2.5
0,2019-01-01:01,41.0,24.0
1,2019-01-01:02,42.0,31.0
2,2019-01-01:03,43.0,29.0
3,2019-01-01:04,46.0,33.0
4,2019-01-01:05,48.0,37.0


In [57]:
df1.loc[0]['pm10']

41.0

In [63]:
df1.loc[1]['pm10']

42.0

In [58]:
temp=[]
for i in range(len(df1)):
    if i%24 == 0:
        temp.append(df1.loc[i]['pm10'])

In [55]:
temp

[48.0,
 28.0,
 33.0,
 31.0,
 34.0,
 43.0,
 28.0,
 12.0,
 11.0,
 13.0,
 4.0,
 6.0,
 23.0,
 41.0,
 58.0,
 nan,
 60.0,
 96.0,
 47.0,
 9.0,
 nan,
 44.0,
 18.0,
 21.0,
 31.0,
 58.0,
 26.0,
 19.0,
 17.0,
 16.0,
 25.0]

In [68]:
pm10 = [[0 for _ in range(25)] for _ in range(32)]
for time in range(1,25):
    temp=[0,]
    for i in range(len(df1)):
        if i%24 == time-1:
            temp.append(df1.loc[i]['pm10'])
    for j in range(1, 25):
        pm10[j][time] = temp[j]
pm10[1]

[0,
 41.0,
 42.0,
 43.0,
 46.0,
 48.0,
 45.0,
 44.0,
 30.0,
 33.0,
 32.0,
 36.0,
 36.0,
 46.0,
 44.0,
 46.0,
 42.0,
 36.0,
 34.0,
 31.0,
 36.0,
 31.0,
 30.0,
 24.0,
 20.0]

In [59]:
pm10 = [[0 for _ in range(25)] for _ in range(32)]

In [62]:
for i in range(len(temp)):
    pm10[i][1] = temp[i]
len(pm10[1]) # 1시

25

In [54]:
# 1시 ~ 24시
# pm[1] ~ pm[24]
# pm10[1][1]
# pm10['날짜']['시']

28.0